In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import datetime

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
PREPROCESSED_DIR = r"C:\project\floodnet-semisupervised-vqa\preprocessed_classification"
MODELS_DIR = r"C:\project\floodnet-semisupervised-vqa\models"
LOGS_DIR = r"C:\project\floodnet-semisupervised-vqa\logs"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 30
STEPS_PER_EPOCH = 20  # As mentioned in the paper
LEARNING_RATE = 0.001  # As mentioned in the paper

In [ ]:
# Create necessary directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

In [ ]:
# Define model building function - following exactly the paper description
def build_inceptionv3_model():
    """
    Build InceptionV3 model as described in the paper:
    - InceptionV3 base with ImageNet weights
    - Global Average Pooling
    - Fully connected layer with 1024 neurons and ReLU activation
    - Output layer with 2 neurons and softmax activation
    """
    # Load pre-trained InceptionV3 model without top layers
    base_model = InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Add classification layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)  # 2 classes: Flooded and Non-Flooded
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile model with Adam optimizer and binary cross entropy loss
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Define model building function for ResNet50 (alternative)
def build_resnet50_model():
    """Build ResNet50 model as described in the paper"""
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Define model building function for Xception (alternative)
def build_xception_model():
    """Build Xception model as described in the paper"""
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Data generator setup with augmentation
def setup_data_generators():
    """Set up data generators for training and validation"""
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    # Validation data generator (only rescaling)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Test data generator (only rescaling)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load training data
    train_generator = train_datagen.flow_from_directory(
        os.path.join(PREPROCESSED_DIR, 'train'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    
    # We don't have labeled validation data in the challenge dataset
    # For internal validation, we'll use a separate validation set
    # This would need to be adjusted based on your available validation labels
    val_generator = val_datagen.flow_from_directory(
        os.path.join(PREPROCESSED_DIR, 'train'),  # using train for validation (would need to split in real scenario)
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    
    return train_generator, val_generator

In [ ]:
# Semi-supervised learning with pseudo-labeling
def generate_pseudo_labels(model, confidence_threshold=0.85):
    """
    Generate pseudo-labels for unlabeled data 
    using the model's predictions with high confidence
    """
    print("Generating pseudo-labels for unlabeled images...")
    
    # Directory for unlabeled images
    unlabeled_dir = os.path.join(PREPROCESSED_DIR, 'train_unlabeled')
    
    # Directories for pseudo-labeled images
    pseudo_flooded_dir = os.path.join(PREPROCESSED_DIR, 'pseudo_labeled', 'Flooded')
    pseudo_nonflooded_dir = os.path.join(PREPROCESSED_DIR, 'pseudo_labeled', 'Non-Flooded')
    
    # Create directories if they don't exist
    os.makedirs(pseudo_flooded_dir, exist_ok=True)
    os.makedirs(pseudo_nonflooded_dir, exist_ok=True)
    
    # Get all unlabeled images
    unlabeled_images = [f for f in os.listdir(unlabeled_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Counter for pseudo-labels
    flooded_count = 0
    nonflooded_count = 0
    skipped_count = 0
    
    # Process each unlabeled image
    for img_name in tqdm(unlabeled_images):
        img_path = os.path.join(unlabeled_dir, img_name)
        
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Get prediction
        prediction = model.predict(img_array)[0]
        confidence = max(prediction)
        predicted_class = np.argmax(prediction)
        
        # Only keep high confidence predictions
        if confidence >= confidence_threshold:
            if predicted_class == 0:  # Non-Flooded
                target_dir = pseudo_nonflooded_dir
                nonflooded_count += 1
            else:  # Flooded
                target_dir = pseudo_flooded_dir
                flooded_count += 1
                
            # Copy the image to the appropriate pseudo-label directory
            from shutil import copy
            copy(img_path, os.path.join(target_dir, img_name))
        else:
            skipped_count += 1
    
    print(f"Pseudo-labeling complete:")
    print(f"  - Flooded: {flooded_count}")
    print(f"  - Non-Flooded: {nonflooded_count}")
    print(f"  - Skipped (low confidence): {skipped_count}")
    
    return flooded_count + nonflooded_count

In [ ]:
# Training function
def train_initial_model(model_name='inceptionv3'):
    """Train the initial model on labeled data"""
    print(f"Training initial {model_name} model on labeled data...")
    
    # Set up data generators
    train_generator, val_generator = setup_data_generators()
    
    # Build model based on selected architecture
    if model_name.lower() == 'inceptionv3':
        model = build_inceptionv3_model()
    elif model_name.lower() == 'resnet50':
        model = build_resnet50_model()
    elif model_name.lower() == 'xception':
        model = build_xception_model()
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    # Set up callbacks
    callbacks = [
        ModelCheckpoint(
            os.path.join(MODELS_DIR, f'{model_name}_initial.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            patience=10,
            monitor='val_accuracy',
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        ),
        TensorBoard(
            log_dir=os.path.join(LOGS_DIR, f'{model_name}_initial_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
        )
    ]
    
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH,  # As specified in the paper
        epochs=EPOCHS,  # As specified in the paper
        validation_data=val_generator,
        validation_steps=STEPS_PER_EPOCH // 4,  # Adjust based on validation set size
        callbacks=callbacks
    )
    
    # Save final model
    model.save(os.path.join(MODELS_DIR, f'{model_name}_final.h5'))
    
    # Plot training history
    plot_training_history(history, model_name)
    
    return model, history

In [ ]:
# Train with semi-supervised learning
def train_with_pseudo_labels(initial_model, model_name='inceptionv3'):
    """Train model with pseudo-labels in a semi-supervised approach"""
    print("Starting semi-supervised training with pseudo-labels...")
    
    # Generate pseudo-labels using the initial model
    pseudo_label_count = generate_pseudo_labels(initial_model)
    
    if pseudo_label_count == 0:
        print("No confident pseudo-labels generated. Skipping semi-supervised training.")
        return initial_model, None
    
    # Set up new data generator including pseudo-labeled data
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    # Load combined training data (original + pseudo-labeled)
    combined_train_generator = train_datagen.flow_from_directory(
        os.path.join(PREPROCESSED_DIR, 'combined_train'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    
    # Validation data generator
    val_datagen = ImageDataGenerator(rescale=1./255)
    val_generator = val_datagen.flow_from_directory(
        os.path.join(PREPROCESSED_DIR, 'train'),  # using train for validation (would need to split in real scenario)
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    
    # Create a fresh model
    if model_name.lower() == 'inceptionv3':
        model = build_inceptionv3_model()
    elif model_name.lower() == 'resnet50':
        model = build_resnet50_model()
    elif model_name.lower() == 'xception':
        model = build_xception_model()
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    # Set up callbacks
    callbacks = [
        ModelCheckpoint(
            os.path.join(MODELS_DIR, f'{model_name}_semisupervised.h5'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            patience=10,
            monitor='val_accuracy',
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        ),
        TensorBoard(
            log_dir=os.path.join(LOGS_DIR, f'{model_name}_semisupervised_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
        )
    ]
    
    # Train the model with combined data
    history = model.fit(
        combined_train_generator,
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=STEPS_PER_EPOCH // 4,
        callbacks=callbacks
    )
    
    # Save final model
    model.save(os.path.join(MODELS_DIR, f'{model_name}_semisupervised_final.h5'))
    
    # Plot training history
    plot_training_history(history, f"{model_name}_semisupervised")
    
    return model, history


In [ ]:
# Prepare combined training set with original labels and pseudo-labels
def prepare_combined_training_set():
    """Prepare a combined training set with original and pseudo-labeled data"""
    # Define paths
    original_train = os.path.join(PREPROCESSED_DIR, 'train')
    pseudo_labeled = os.path.join(PREPROCESSED_DIR, 'pseudo_labeled')
    combined_train = os.path.join(PREPROCESSED_DIR, 'combined_train')
    
    # Create combined directory
    os.makedirs(os.path.join(combined_train, 'Flooded'), exist_ok=True)
    os.makedirs(os.path.join(combined_train, 'Non-Flooded'), exist_ok=True)
    
    # Copy original labeled data
    import shutil
    
    # Copy Flooded images
    original_flooded = os.path.join(original_train, 'Flooded')
    combined_flooded = os.path.join(combined_train, 'Flooded')
    for img in os.listdir(original_flooded):
        if img.endswith(('.jpg', '.jpeg', '.png')):
            shutil.copy(
                os.path.join(original_flooded, img),
                os.path.join(combined_flooded, f"orig_{img}")
            )
    
    # Copy Non-Flooded images
    original_nonflooded = os.path.join(original_train, 'Non-Flooded')
    combined_nonflooded = os.path.join(combined_train, 'Non-Flooded')
    for img in os.listdir(original_nonflooded):
        if img.endswith(('.jpg', '.jpeg', '.png')):
            shutil.copy(
                os.path.join(original_nonflooded, img),
                os.path.join(combined_nonflooded, f"orig_{img}")
            )
    
    # Copy pseudo-labeled Flooded images
    pseudo_flooded = os.path.join(pseudo_labeled, 'Flooded')
    if os.path.exists(pseudo_flooded):
        for img in os.listdir(pseudo_flooded):
            if img.endswith(('.jpg', '.jpeg', '.png')):
                shutil.copy(
                    os.path.join(pseudo_flooded, img),
                    os.path.join(combined_flooded, f"pseudo_{img}")
                )
    
    # Copy pseudo-labeled Non-Flooded images
    pseudo_nonflooded = os.path.join(pseudo_labeled, 'Non-Flooded')
    if os.path.exists(pseudo_nonflooded):
        for img in os.listdir(pseudo_nonflooded):
            if img.endswith(('.jpg', '.jpeg', '.png')):
                shutil.copy(
                    os.path.join(pseudo_nonflooded, img),
                    os.path.join(combined_nonflooded, f"pseudo_{img}")
                )
    
    # Print statistics
    flooded_count = len([f for f in os.listdir(combined_flooded) if f.endswith(('.jpg', '.jpeg', '.png'))])
    nonflooded_count = len([f for f in os.listdir(combined_nonflooded) if f.endswith(('.jpg', '.jpeg', '.png'))])
    
    print(f"Combined training set prepared:")
    print(f"  - Flooded: {flooded_count} images")
    print(f"  - Non-Flooded: {nonflooded_count} images")
    print(f"  - Total: {flooded_count + nonflooded_count} images")
    
    return flooded_count + nonflooded_count

In [ ]:
# Utility functions
def plot_training_history(history, model_name):
    """Plot and save training history"""
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='validation')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='validation')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(LOGS_DIR, f'{model_name}_training_history.png'))
    plt.close()

In [ ]:
def evaluate_model(model, dataset_name='validation'):
    """Evaluate model on validation or test set"""
    print(f"Evaluating model on {dataset_name} set...")
    
    # Load data generator
    datagen = ImageDataGenerator(rescale=1./255)
    
    # We need ground truth labels to evaluate properly
    # For demonstration, assuming we have validation labels
    generator = datagen.flow_from_directory(
        os.path.join(PREPROCESSED_DIR, dataset_name),
        target_size=IMG_SIZE,
        batch_size=1,
        class_mode='categorical',
        shuffle=False
    )
    
    # Get predictions
    predictions = model.predict(generator, steps=len(generator))
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get true labels
    true_classes = generator.classes
    
    # Class mapping
    class_indices = generator.class_indices
    class_names = {v: k for k, v in class_indices.items()}
    
    # Print classification report
    report = classification_report(
        true_classes,
        predicted_classes,
        target_names=list(class_names.values())
    )
    print(report)
    
    # Plot confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(class_names.values()),
                yticklabels=list(class_names.values()))
    plt.title(f'Confusion Matrix - {dataset_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(LOGS_DIR, f'confusion_matrix_{dataset_name}.png'))
    
    return report, cm

In [ ]:
# Main execution
if __name__ == "__main__":
    print("FloodNet Image Classification Training")
    print("=====================================")
    
    # Check if preprocessed data exists
    if not os.path.exists(PREPROCESSED_DIR):
        print(f"Error: Preprocessed directory {PREPROCESSED_DIR} does not exist!")
        print("Please run the preprocessing script first.")
        exit(1)
    
    # Choose model
    model_name = 'inceptionv3'  # Options: 'inceptionv3', 'resnet50', 'xception'
    print(f"Selected model: {model_name}")
    
    # Train initial model on labeled data
    initial_model, initial_history = train_initial_model(model_name)
    
    # Generate pseudo-labels and prepare combined dataset
    prepare_combined_training_set()
    
    # Train with semi-supervised approach
    final_model, semisup_history = train_with_pseudo_labels(initial_model, model_name)
    
    # Evaluate model (requires proper validation set with labels)
    try:
        evaluate_model(final_model, 'validation')
    except Exception as e:
        print(f"Evaluation failed: {e}")
        print("Note: To properly evaluate the model, you need labeled validation data.")
    
    print("\nTraining Complete!")
    print(f"Model saved to: {os.path.join(MODELS_DIR, f'{model_name}_semisupervised_final.h5')}")